# Load environement

In [20]:
!pip install -q -U google-generativeai==0.4.1 beautifulsoup4==4.12.2

In [ ]:
!code .

In [1]:
from dotenv import load_dotenv

load_dotenv()

MAX_ITER = 10

import os
import google.generativeai as genai

GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]
genai.configure(api_key=GOOGLE_API_KEY)
generation_config = {
    "stop_sequences": ["<observation>"],
}
model = genai.GenerativeModel("gemini-pro")

# Actions

In [2]:
from typing import Callable


def look_at_dictionary_1(word: str) -> str:
    """Search dictionary 1.
    Args:
        word (str): Word to query
    Returns:
        meaning of the word
    """
    my_dict = {"cacacopoadof": "cat"}
    try:
        return f"The meaning of {word} is {my_dict[word.lower()]}"
    except KeyError:
        return f"The word {word} is not found in this dictionary 1"
    
def look_at_dictionary_2(word: str) -> str:
    """Search dictionary 2.
    Args:
        word (str): Word to query
    Returns:
        meaning of the word
    """
    my_dict = {"papapapipikio": "chicken"}
    try:
        return f"The meaning of {word} is {my_dict[word.lower()]}"
    except KeyError:
        return f"The word {word} is not found in this dictionary 2"


def build_action_prompt(actions: list[Callable]) -> str:
    return "\n".join(
        [
            f"Index: {i} \n" + action.__name__ + "\n" + action.__doc__
            for i, action in enumerate(actions)
        ]
    )


actions = [look_at_dictionary_1, look_at_dictionary_2]
action_prompt = build_action_prompt(actions)
print(action_prompt)


Index: 0 
look_at_dictionary_1
Search dictionary 1.
    Args:
        word (str): Word to query
    Returns:
        meaning of the word
    
Index: 1 
look_at_dictionary_2
Search dictionary 2.
    Args:
        word (str): Word to query
    Returns:
        meaning of the word
    


# Run

In [10]:
import json
from bs4 import BeautifulSoup

question = "what is Papapapipikio and Cacacopoadof?"

prompt = f"""
Human: Answer the following questions as best you can.
You can perform the these below actions to approach the answer.
You should think about all possible inputs to these actions.:

{action_prompt}

<question>the input question you must answer</question>
<answer>
    <step count=number start from 1>
        <thought>you should always think about what to do</thought>
        <action>
        <action_to_take>the index of your selected tool</action_to_take>
        <action_input>the input to the action answer in JSON format</action_input>
        </action>
        <observation>the result of the action</observation>
    </step>
... (Step can repeat N times)
<step count=final>
<thought>I now know the final answer</thought>
<final_answer>the final answer to the original input question</final_answer>
</step>
</answer>
Begin!

<question>{question}</question>
<answer>
"""


for step in range(1, MAX_ITER + 1):
    print(f"Step: {step}")
    # LLM generating given prompt
    response = model.generate_content(prompt, generation_config=generation_config)
    beautiful_soup = BeautifulSoup(response.text)
    if beautiful_soup.find("final_answer"):
        print(response.text)
        break
    
    # Extract thought
    # LLM: According to the question and observation, I think ...
    thought_element = beautiful_soup.find("thought")
    thought = thought_element.text
    print(f"Thought: {thought}")
    
    # Extract action
    # LLM: After thinking here is the action and inputs I choose to approach the answer
    action_element = beautiful_soup.find("action")
    action_to_take = int(action_element.find("action_to_take").text)
    action_input = json.loads(action_element.find("action_input").text)

    # Action!
    output = actions[action_to_take](**action_input)
    print("Output:")
    print(output)
    
    # Append propmt
    step_prompt = f"""
<step count={step}>
    {thought_element}
    {action_element}
    <observation>
        "action output": {output}
    </observation>
</step>
"""
    prompt += step_prompt
    print("-" * 16)


Step: 1
Thought: I can use look_at_dictionary_1 to find the definition of Papapapipikio
Output:
The word Papapapipikio is not found in this dictionary 1
----------------
Step: 2
Thought: I can use look_at_dictionary_2 to find the definition of Papapapipikio
Output:
The meaning of Papapapipikio is chicken
----------------
Step: 3
Thought: I can use look_at_dictionary_1 to find the definition of Cacacopoadof
Output:
The meaning of Cacacopoadof is cat
----------------
Step: 4
<final_answer>Papapapipikio means chicken and Cacacopoadof means cat.</final_answer>
</answer>


In [14]:
# Optional print answer html
!pip install "rich[jupyter]"
from rich.syntax import Syntax



finalize = prompt + response.text
answer = BeautifulSoup(finalize).find_all("answer")[1]
syntax = Syntax(str(answer), "html")
syntax

<answer>                                                                                                           
<step count="1">                                                                                                   
<thought>I can use look_at_dictionary_1 to find the definition of Papapapipikio</thought>                          
<action>                                                                                                           
<action_to_take>0</action_to_take>                                                                                 
<action_input>                                                                                                     
{                                                                                                                  
    "word": "Papapapipikio"                                                                                        
}                                                                                                                  
</action_input>                                                                                                    
</action>                                                                                                          
<observation>                                                                                                      
        "action output": The word Papapapipikio is not found in this dictionary 1                                  
    </observation>                                                                                                 
</step>                                                                                                            
<step count="2">                                                                                                   
<thought>I can use look_at_dictionary_2 to find the definition of Papapapipikio</thought>                          
<action>                                                                                                           
<action_to_take>1</action_to_take>                                                                                 
<action_input>                                                                                                     
{                                                                                                                  
    "word": "Papapapipikio"                                                                                        
}                                                                                                                  
</action_input>                                                                                                    
</action>                                                                                                          
<observation>                                                                                                      
        "action output": The meaning of Papapapipikio is chicken                                                   
    </observation>                                                                                                 
</step>                                                                                                            
<step count="3">                                                                                                   
<thought>I can use look_at_dictionary_1 to find the definition of Cacacopoadof</thought>                           
<action>                                                                                                           
<action_to_take>0</action_to_take>                                                                                 
<action_input>                                                                                                     
{                                                       